# Projet de base

In [1]:
import numpy as np
import math
import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout
from matplotlib import pyplot as plt
import copy
import time

In [2]:
class Node:
    
    def __init__(self,idx, parent, children, lb, visited):
        self.idx = idx
        self.parent = parent
        self.children = children
        self.lb = lb
        self.visited = visited
        
    def __str__(self):
        return ("Parent : " +str(self.parent)+
                "\n Children : "+str(self.children) + 
                "\n lb : "+str(self.lb)+ 
                "\n visited : "+str(self.visited))

In [3]:
def arborescence(node,Tree):
    branch = []
    while node.parent != None:
        branch.append(node)
        node = Tree[node.parent]
    branch.append(node)
    
    return branch

In [4]:
def computeCost(p,d,w,order):
    t=0
    cost=0
    for idx in order:
        t += p[idx]
        cost += max(0,(t-d[idx])*w[idx])
    return cost

In [5]:
p = [12,8,15,9]
d = [16,26,25,27]
w = [4,5,3,5]

In [6]:
def getPrimal_0(p,d,w):
    sortedTasks = np.argsort(d)
    cost = computeCost(p,d,w,sortedTasks)
    return(cost,sortedTasks)


def getPrimal_1(p,d,w):
    sortedTasks = (np.argsort((np.array(d)-np.array(p))*np.array(w)))[::-1]
    cost = computeCost(p,d,w,sortedTasks)
    return(cost,sortedTasks)


def getPrimal_2(p,d,w):
    tasksLeft = [int(i) for i in range(len(p))]
    order = []
    while len(tasksLeft)!= 0:
        time = np.sum(np.array(p)[tasksLeft])
        penalities = (time-np.array(d)[tasksLeft])*np.array(w)[tasksLeft]
        sortedTasks = np.argsort(penalities)
        order.append(tasksLeft[sortedTasks[0]])
        tasksLeft.pop(sortedTasks[0])
    order.reverse()
    cost = computeCost(p,d,w,order)
    return(cost,order)

In [7]:
def getDual_0(p,d,w,visited):
    dual=0
    time=0
    notVisited=list(range(len(p)))
    for i in visited:
        retard=((p[i]+time)-d[i])
        dual+=retard*w[i]*(retard>0)
        time+=p[i]
        notVisited.remove(i)
        
    for j in notVisited:
        retard=(p[j]+time)-d[j]
        dual+=retard*w[j]*(retard>0)
    return dual

def getDual_1(p,d,w,visited):
    dual = 0
    time = 0
    notVisited = list(range(len(p)))
    for i in visited:
        retard = ((p[i]+time)-d[i])
        dual += retard*w[i]*(retard>0)
        time += p[i]
        notVisited.remove(i)
    return dual

def getDual_2(p,d,w,visited):
    dual = 0
    time = np.sum(p)
    for i in visited:
        retard = (time-d[i])
        dual += retard*w[i]*(retard>0)
        time -= p[i] 
    return dual

def getDual_3(p,d,w,visited):
    dual = 0
    time = np.sum(p)
    notVisited=list(range(len(p)))
    for i in visited:
        retard = (time-d[i])
        dual += retard*w[i]*(retard>0)
        time -= p[i] 
        notVisited.remove(i)
    for j in notVisited:
        if time - p[j] < d[j]:
            retard = time-d[j]
            dual+=retard*w[j]*(retard>0)
    return dual

In [8]:
def exploration_0(Queue,Tree,UB):
    return(Queue[0])

def exploration_1(Queue,Tree,UB):
    return(Queue[-1])

def exploration_2(Queue,Tree,UB):
    bestNode = Queue[-1]
    bestLB = -math.inf
    for idx in Queue:
        if Tree[idx].lb > bestLB and Tree[idx].lb < UB:
            bestLB = Tree[idx].lb
            bestNode = idx
    return(bestNode)

In [9]:
def branch_and_bound(p,d,w,byLast,getPrimal,getDual,exploration):
    
    Tasks = list(range(len(p)))
    Tree = [] # list of all nodes created (a list of Node objects)
    Queue = [] # list of nodes to process (a list of integers with the index of nodes to process in the Tree)
    UB = 10000000 # Upper Bound of the problem
    LB = 0 # best Lower Bound of the problem 
    
    UB,currentOrder = getPrimal(p,d,w) #We start by computing the first primal/upper bound.

    bestNode = Node(-1,-1, [], 0 ,[]) #default best node
    
    root = Node(0,None, [], 0, []) # define the root node of the tree
    Tree.append(root) # start the tree with the root node
    Queue.append(0) # start the list of nodes to process with the root node
    
    #continue processing nodes until the queue is empty
    
    nbIterations = 0
    
    while len(Queue)!=0 :
        
        # process the nodes in the queue according to the chosen exploration method
        
        currentIndex = exploration(Queue,Tree,UB)
        currentNode = Tree[currentIndex]
        nbIterations += 1
        
        if currentNode.lb < UB:
            
            # If the processed node corresponds to a "full-schedule" we check whether this solution is an improvement or not
            # If so, we update the problem Upper Bound / best order / best node
            
            if len(currentNode.visited) == len(p):
                order = copy.copy(currentNode.visited)
                if byLast:
                    order.reverse()
                solutionCost = computeCost(p,d,w,order)
                if solutionCost <= UB:
                    UB = solutionCost
                    currentOrder = order
                    bestNode = currentNode
            else:
                
                #for each task not processed yet we create a new node
                for nextTask in (set(Tasks)-set(currentNode.visited)):
                    nextVisited = copy.copy(currentNode.visited)
                    nextVisited.append(nextTask)
                    nextLb = getDual(p,d,w, nextVisited)

                    newNode = Node(len(Tree),currentIndex, [], nextLb,nextVisited)
                    Tree.append(newNode)
                    Queue.append(len(Tree)-1)
                    currentNode.children.append(len(Tree)-1)
            
        # When the processing of the node is completed remove the node from the queue     
        Queue.remove(currentIndex)

    # return the  solution
    
    if (UB-bestNode.lb)<=0.0001:
        branch = arborescence(bestNode,Tree)
    
    # If the best node is not a leaf, the bestNode variable is still the default one default
    else:
        bestLB = -math.inf
        for node in Tree:
            if node.lb > bestLB and node.lb <=UB:
                bestLB = node.lb
                bestNode=node
        branch = arborescence(bestNode,Tree)

    return UB, currentOrder, nbIterations, branch, Tree

# Extension

Nous allons maintenant considérer une extension de notre problème dans lequel l'atelier dispose de plusieurs machines : 
Nous ne changeons pas la structure du code, nous nous appuierons toujours sur la classe Node, les fonctions _getPrimal_, _getDual_, _exploration_ et sur la fonction _Branch and Bound_.


Le changement majeur est le changement de format de l'attribut _visited_ de chacun de nos noeuds. En effet, auparavant, cet attribut était une liste stockant les tâches déja réalisées (visted = [0,4,7] <=> les tâches 0,4,7 sont éffectuées, que ce soit en partant de la fin ou du début.  
visited : [0,2,3] -> [[0,2],[3]]

In [10]:
class Node:
    
    def __init__(self,idx, parent, children, lb, visited):
        self.idx = idx
        self.parent = parent
        self.children = children
        self.lb = lb
        self.visited = visited
        
    def __str__(self):
        return ("Parent : " +str(self.parent)+
                "\n Children : "+str(self.children) + 
                "\n lb : "+str(self.lb)+ 
                "\n visited : "+str(self.visited))

In [11]:
def computeCost(p,d,w,orders,n):
    cost=0
    for order in orders:
        t=0
        for idx in order:
            t += p[idx]
            cost += max(0,(t-d[idx])*w[idx])
    return cost

In [12]:
def getPrimal_2(p,d,w,n):
    tasksLeft = [int(i) for i in range(len(p))]
    orders = []
    while len(tasksLeft)!= 0:
        time = np.sum(np.array(p)[tasksLeft])
        penalities = (time-np.array(d)[tasksLeft])*np.array(w)[tasksLeft]
        sortedTasks = np.argsort(penalities)
        order.append(tasksLeft[sortedTasks[0]])
        tasksLeft.pop(sortedTasks[0])
    order.reverse()
    cost = computeCost(p,d,w,orders,n)
    return(cost,order)

In [13]:
def getPrimal_2(p,d,w,n):
    tasksLeft = [int(i) for i in range(len(p))]
    orders = [[] for _ in range(n)]
    time = [np.sum(np.array(p)[tasksLeft])]*n
    
    while len(tasksLeft)!= 0:
        bestPenalty = (1e99,0,0)
        for k in range(n):
            penalties = (time[k]-np.array(d)[tasksLeft])*np.array(w)[tasksLeft]
            sortedTasks = np.argsort(penalties)
            if sortedTasks[0]<bestPenalty[0]:
                bestPenalty = (sortedTasks[0], k, tasksLeft[sortedTasks[0]]) #Penalty / machine / task
        
        orders[bestPenalty[1]].append(bestPenalty[2]) #orders[machine].append(task)
        time[bestPenalty[1]] -= p[bestPenalty[2]]
        tasksLeft.pop(sortedTasks[0])
    for order in orders:
        order.reverse()
    order.reverse()
    cost = computeCost(p,d,w,orders,n)
    return(cost,order)

In [14]:
def getPrimal(p,d,w,n):
    sortedTasks = np.argsort(d)
    orders = [[] for _ in range(n)]
    for i in range(len(sortedTasks)):
        orders[i%n].append(sortedTasks[i])
    cost = computeCost(p,d,w,orders,n)
    return(cost,orders)

In [15]:
def getDual(p,d,w,visited,n):
    dual = 0
    time = [np.sum(p)]*n #-> [90,90,90] pour n = 3 par exemple
    for k in range(n):
        machine = visited[k]
        for i in machine:
            retard = (time[k]-d[i])
            dual += retard*w[i]*(retard>0)
            time[k] -= p[i] 
    return dual

In [16]:
def getDual_2(p,d,w,visited,n):
    dual = 0
    time = [0]*n 
    for k in range(n):
        machine = visited[k]
        for i in machine:
            retard = (time[k] + p[i] -d[i])
            dual += retard*w[i]*(retard>0)
            time[k] += p[i] 
    return dual

In [17]:
def exploration_0(Queue,Tree,UB):
    return(Queue[0])

def exploration_1(Queue,Tree,UB):
    return(Queue[-1])

In [23]:
def branch_and_bound(p,d,w,byLast,getPrimal,getDual,exploration,n):
    
    Tasks = list(range(len(p)))
    Tree = [] # list of all nodes created (a list of Node objects)
    Queue = [] # list of nodes to process (a list of integers with the index of nodes to process in the Tree)
    UB = 10000000 # Upper Bound of the problem
    LB = 0 # best Lower Bound of the problem 
    
    UB,currentOrder = getPrimal(p,d,w,n) #We start by computing the first primal/upper bound.
    print("Primal Bound : {} | Order : {}".format(UB,currentOrder))
    bestNode = Node(-1,-1, [], 0 ,[]) #default best node
    
    root = Node(0,None, [], 0, [[] for _ in range(n)]) # define the root node of the tree
    Tree.append(root) # start the tree with the root node
    Queue.append(0) # start the list of nodes to process with the root node
    
    #continue processing nodes until the queue is empty
    
    nbIterations = 0
    
    while len(Queue)!=0 :
        
        # process the nodes in the queue according to the chosen exploration method
        
        currentIndex = exploration(Queue,Tree,UB)
        currentNode = Tree[currentIndex]
        nbIterations += 1
        
        print(currentNode)
        if currentNode.lb < UB:
            
            # If the processed node corresponds to a "full-schedule" we check whether this solution is an improvement or not
            # If so, we update the problem Upper Bound / best order / best node
            
            if sum([len(visited) for visited in currentNode.visited]) >= len(p):
                orders = copy.copy(currentNode.visited)
                if byLast:
                    for order in orders:
                        order.reverse()
                solutionCost = computeCost(p,d,w,orders)
                if solutionCost <= UB:
                    UB = solutionCost
                    currentOrder = orders
                    bestNode = currentNode
            else:
                
                #for each task not processed yet, for every machine, we create a new node
                allVisitedTasks = []
                for machine in currentNode.visited:
                    for task in machine:
                        allVisitedTasks.append(task)
                        
                for nextTask in (set(Tasks)-set(allVisitedTasks)):
                    for k in range(n):
                        nextVisited = copy.copy(currentNode.visited)
                        nextVisited[k].append(nextTask)
                        nextLb = getDual(p,d,w, nextVisited,n)

                        newNode = Node(len(Tree),currentIndex, [], nextLb,nextVisited)
                        Tree.append(newNode)
                        Queue.append(len(Tree)-1)
                        currentNode.children.append(len(Tree)-1)
            
        # When the processing of the node is completed remove the node from the queue     
        Queue.remove(currentIndex)

    # return the  solution
    
    if (UB-bestNode.lb)<=0.0001:
        branch = arborescence(bestNode,Tree)
    
    # If the best node is not a leaf, the bestNode variable is still the default one default
    else:
        bestLB = -math.inf
        for node in Tree:
            if node.lb > bestLB and node.lb <=UB:
                bestLB = node.lb
                bestNode=node
        branch = arborescence(bestNode,Tree)

    return UB, currentOrder, nbIterations, branch, Tree

In [24]:
p = [2,3,4,2,3,6]
d = [2,5,9,2,5,7]
w = [1,1,1,1,1,1]

orders = [[0,1,2],[3,4,5]]
computeCost(p,d,w,orders,2)

4

In [25]:
sol, order, _ ,_ ,_= branch_and_bound(p,d,w,True,getPrimal,getDual,exploration_1,2)
print("Solution :", sol)
print("Ordre : ", order)

Primal Bound : 4 | Order : [[0, 1, 5], [3, 4, 2]]
Parent : None
 Children : []
 lb : 0
 visited : [[], []]
Parent : 0
 Children : []
 lb : 100
 visited : [[0, 1, 2, 3, 4, 5], [0, 1, 2, 3, 4, 5]]
Parent : 0
 Children : []
 lb : 100
 visited : [[0, 1, 2, 3, 4, 5], [0, 1, 2, 3, 4, 5]]
Parent : 0
 Children : []
 lb : 100
 visited : [[0, 1, 2, 3, 4, 5], [0, 1, 2, 3, 4, 5]]
Parent : 0
 Children : []
 lb : 96
 visited : [[0, 1, 2, 3, 4, 5], [0, 1, 2, 3, 4, 5]]
Parent : 0
 Children : []
 lb : 92
 visited : [[0, 1, 2, 3, 4, 5], [0, 1, 2, 3, 4, 5]]
Parent : 0
 Children : []
 lb : 83
 visited : [[0, 1, 2, 3, 4, 5], [0, 1, 2, 3, 4, 5]]
Parent : 0
 Children : []
 lb : 74
 visited : [[0, 1, 2, 3, 4, 5], [0, 1, 2, 3, 4, 5]]
Parent : 0
 Children : []
 lb : 68
 visited : [[0, 1, 2, 3, 4, 5], [0, 1, 2, 3, 4, 5]]
Parent : 0
 Children : []
 lb : 62
 visited : [[0, 1, 2, 3, 4, 5], [0, 1, 2, 3, 4, 5]]
Parent : 0
 Children : []
 lb : 49
 visited : [[0, 1, 2, 3, 4, 5], [0, 1, 2, 3, 4, 5]]
Parent : 0
 Children

In [26]:
def genMultiInstances(n,alpha,n_machines):
    p=np.random.randint(1,15,n)
    d=np.random.randint(3,int(alpha*n*15/n_machines),n)
    w=np.random.randint(1,5,n)
    return(p,d,w)

In [27]:
for _ in range(3):
    print("______________________")
    p,d,w = genMultiInstances(12,.4,2)
    print("p : ", p)
    print("d : ", d)
    print("w : ", w)
    sol, order, nbit ,_ ,_= branch_and_bound(p,d,w,True,getPrimal,getDual,exploration_1,2)
    sol2, order2, nbit2 ,_ ,_= branch_and_bound(p,d,w,False,getPrimal,getDual_2,exploration_1,2)
    print("Solution : {} | Ordre : {} | nbit : {}".format(sol,order,nbit))
    print("Solution2: {} | Ordre2: {} | nbit2: {}".format(sol2,order2,nbit2))

______________________
p :  [ 2  4  5  2 11  2  1  4 14  5  8 13]
d :  [19 12 34 29 28 32 23 17 24 11 33 30]
w :  [4 4 1 1 2 2 1 4 4 3 4 1]
Primal Bound : 40 | Order : [[9, 7, 6, 4, 11, 10], [1, 0, 8, 3, 5, 2]]
Parent : None
 Children : []
 lb : 0
 visited : [[], []]
Parent : 0
 Children : []
 lb : 1654
 visited : [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]]
Parent : 0
 Children : []
 lb : 1654
 visited : [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]]
Parent : 0
 Children : []
 lb : 1654
 visited : [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]]
Parent : 0
 Children : []
 lb : 1654
 visited : [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]]
Parent : 0
 Children : []
 lb : 1654
 visited : [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]]
Parent : 0
 Children : []
 lb : 1609
 visited : [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

TypeError: computeCost() missing 1 required positional argument: 'n'